In [1]:
import numpy as np
import xarray as xr
import dask.array as dsar
import xrft

In [2]:
def _synthetic_field(N, dL, amp, s):
    """
    Generate a synthetic series of size N by N
    with a spectral slope of s.
    """

    k = np.fft.fftshift(np.fft.fftfreq(N, dL))
    l = np.fft.fftshift(np.fft.fftfreq(N, dL))
    kk, ll = np.meshgrid(k, l)
    K = np.sqrt(kk**2+ll**2)

    ########
    # amplitude
    ########
    r_kl = np.ma.masked_invalid(np.sqrt(amp*.5*(np.pi)**(-1)
                                *K**(s-1.))).filled(0.)
    ########
    # phase
    ########
    phi = np.zeros((N, N))

    N_2 = int(N/2)
    phi_upper_right = 2.*np.pi*np.random.random((N_2-1,
                                                 N_2-1)) - np.pi
    phi[N_2+1:,N_2+1:] = phi_upper_right.copy()
    phi[1:N_2, 1:N_2] = -phi_upper_right[::-1, ::-1].copy()


    phi_upper_left = 2.*np.pi*np.random.random((N_2-1,
                                                N_2-1)) - np.pi
    phi[N_2+1:,1:N_2] = phi_upper_left.copy()
    phi[1:N_2, N_2+1:] = -phi_upper_left[::-1, ::-1].copy()


    phi_upper_middle = 2.*np.pi*np.random.random(N_2) - np.pi
    phi[N_2:, N_2] = phi_upper_middle.copy()
    phi[1:N_2, N_2] = -phi_upper_middle[1:][::-1].copy()


    phi_right_middle = 2.*np.pi*np.random.random(N_2-1) - np.pi
    phi[N_2, N_2+1:] = phi_right_middle.copy()
    phi[N_2, 1:N_2] = -phi_right_middle[::-1].copy()


    phi_edge_upperleft = 2.*np.pi*np.random.random(N_2) - np.pi
    phi[N_2:, 0] = phi_edge_upperleft.copy()
    phi[1:N_2, 0] = -phi_edge_upperleft[1:][::-1].copy()


    phi_bot_right = 2.*np.pi*np.random.random(N_2) - np.pi
    phi[0, N_2:] = phi_bot_right.copy()
    phi[0, 1:N_2] = -phi_bot_right[1:][::-1].copy()


    phi_corner_leftbot = 2.*np.pi*np.random.random() - np.pi

    for i in range(1, N_2):
        for j in range(1, N_2):
            assert (phi[N_2+j, N_2+i] == -phi[N_2-j, N_2-i])

    for i in range(1, N_2):
        for j in range(1, N_2):
            assert (phi[N_2+j, N_2-i] == -phi[N_2-j, N_2+i])

    for i in range(1, N_2):
        assert (phi[N_2, N-i] == -phi[N_2, i])
        assert (phi[N-i, N_2] == -phi[i, N_2])
        assert (phi[N-i, 0] == -phi[i, 0])
        assert (phi[0, i] == -phi[0, N-i])
    #########
    # complex fourier amplitudes
    #########
    F_theta = r_kl * np.exp(1j * phi)

    # check that symmetry of FT is satisfied
    theta = np.fft.ifft2(np.fft.ifftshift(F_theta))
    return np.real(theta)

In [3]:
def _synthetic_field_xr(N, dL, amp, s,
                    other_dim_sizes=None, dim_order=True, 
                    chunks=None):
    
    theta = xr.DataArray(_synthetic_field(N, dL, amp, s),
                        dims=['y', 'x'],
                        coords={'y':range(N), 'x':range(N)}
                        )

    if other_dim_sizes:
        _da = xr.DataArray(np.ones(other_dim_sizes),
                           dims=['d%d'%i for i in range(len(other_dim_sizes))])
        if dim_order:
            theta = theta + _da
        else:
            theta = _da + theta
            
    if chunks:
        theta = theta.chunk(chunks)

    return theta

In [4]:
def test_isotropic_ps_slope(N=512, dL=1., amp=1e1, s=-3., **kwargs):
    """Test the spectral slope of isotropic power spectrum."""

    theta = _synthetic_field_xr(N, dL, amp, s, **kwargs)
    
    #return theta
    iso_ps = xrft.isotropic_power_spectrum(theta, dim=['y','x'],
                                          detrend='constant',
                                          density=True)
    return iso_ps, theta

#test_isotropic_ps_slope(chunks={'y': None, 'x': None, 'd0': 2}, other_dim_sizes=[10], dim_order=True) # breaks
ps, theta = test_isotropic_ps_slope(other_dim_sizes=[10], dim_order=True) # breaks

/Users/aponte/.miniconda3/envs/equinox/lib/python3.7/site-packages/ipykernel_launcher.py:16: RuntimeWarning: divide by zero encountered in power
  app.launch_new_instance()


In [5]:
ps

<xarray.DataArray (d0: 10, freq_r: 127)>
array([[1.82531621e+05, 6.66205642e+03, 7.46340700e+02, ...,
        1.06886504e-04, 1.03539000e-04, 1.00277579e-04],
       [1.82531621e+05, 6.66205642e+03, 7.46340700e+02, ...,
        1.06886504e-04, 1.03539000e-04, 1.00277579e-04],
       [1.82531621e+05, 6.66205642e+03, 7.46340700e+02, ...,
        1.06886504e-04, 1.03539000e-04, 1.00277579e-04],
       ...,
       [1.82531621e+05, 6.66205642e+03, 7.46340700e+02, ...,
        1.06886504e-04, 1.03539000e-04, 1.00277579e-04],
       [1.82531621e+05, 6.66205642e+03, 7.46340700e+02, ...,
        1.06886504e-04, 1.03539000e-04, 1.00277579e-04],
       [1.82531621e+05, 6.66205642e+03, 7.46340700e+02, ...,
        1.06886504e-04, 1.03539000e-04, 1.00277579e-04]])
Coordinates:
  * freq_r          (freq_r) float64 0.002653 0.006041 0.009854 ... 0.4921 0.496
    freq_y_spacing  float64 0.001953
    freq_x_spacing  float64 0.001953
Dimensions without coordinates: d0

In [6]:
test_isotropic_ps_slope()

/Users/aponte/.miniconda3/envs/equinox/lib/python3.7/site-packages/ipykernel_launcher.py:16: RuntimeWarning: divide by zero encountered in power
  app.launch_new_instance()


(<xarray.DataArray (freq_r: 127)>
 array([1.82531621e+05, 6.66205642e+03, 7.46340700e+02, 1.85415150e+02,
        6.59866268e+01, 2.84687453e+01, 1.45499124e+01, 8.24404936e+00,
        4.95285200e+00, 3.15117626e+00, 2.12359625e+00, 1.46886641e+00,
        1.05014553e+00, 7.75269749e-01, 5.81121174e-01, 4.44074919e-01,
        3.45883090e-01, 2.72326603e-01, 2.18550468e-01, 1.77939705e-01,
        1.45682790e-01, 1.20141452e-01, 9.98975311e-02, 8.38687541e-02,
        7.12682207e-02, 6.07114287e-02, 5.19097010e-02, 4.48590834e-02,
        3.89114749e-02, 3.38112091e-02, 2.95755055e-02, 2.60685275e-02,
        2.30129349e-02, 2.03469481e-02, 1.80665040e-02, 1.61202089e-02,
        1.44481492e-02, 1.29850609e-02, 1.16773175e-02, 1.05329533e-02,
        9.52931610e-03, 8.64517406e-03, 7.86062778e-03, 7.16582920e-03,
        6.54811536e-03, 5.98862419e-03, 5.48264420e-03, 5.03752620e-03,
        4.63888408e-03, 4.27930650e-03, 3.95016010e-03, 3.64667042e-03,
        3.37461400e-03, 3.1317

In [7]:
test_isotropic_ps_slope(chunks={'y': None, 'x': None})

/Users/aponte/.miniconda3/envs/equinox/lib/python3.7/site-packages/ipykernel_launcher.py:16: RuntimeWarning: divide by zero encountered in power
  app.launch_new_instance()


(<xarray.DataArray 'rechunk-merge-2164fc1ea9b434b9b8fee016fe9bde8e' (freq_r: 127)>
 dask.array<getitem, shape=(127,), dtype=float64, chunksize=(1,), chunktype=numpy.ndarray>
 Coordinates:
   * freq_r          (freq_r) float64 0.002653 0.006041 0.009854 ... 0.4921 0.496
     freq_y_spacing  float64 0.001953
     freq_x_spacing  float64 0.001953,
 <xarray.DataArray (y: 512, x: 512)>
 dask.array<xarray-<this-array>, shape=(512, 512), dtype=float64, chunksize=(512, 512), chunktype=numpy.ndarray>
 Coordinates:
   * y        (y) int64 0 1 2 3 4 5 6 7 8 ... 503 504 505 506 507 508 509 510 511
   * x        (x) int64 0 1 2 3 4 5 6 7 8 ... 503 504 505 506 507 508 509 510 511)

In [8]:
test_isotropic_ps_slope(other_dim_sizes=[2], chunks={'d0': 2})

/Users/aponte/.miniconda3/envs/equinox/lib/python3.7/site-packages/ipykernel_launcher.py:16: RuntimeWarning: divide by zero encountered in power
  app.launch_new_instance()


(<xarray.DataArray 'rechunk-merge-8cab963273e4ded25ecb98172abfa2f5' (d0: 2, freq_r: 127)>
 dask.array<getitem, shape=(2, 127), dtype=float64, chunksize=(2, 1), chunktype=numpy.ndarray>
 Coordinates:
   * freq_r          (freq_r) float64 0.002653 0.006041 0.009854 ... 0.4921 0.496
     freq_y_spacing  float64 0.001953
     freq_x_spacing  float64 0.001953
 Dimensions without coordinates: d0,
 <xarray.DataArray (y: 512, x: 512, d0: 2)>
 dask.array<xarray-<this-array>, shape=(512, 512, 2), dtype=float64, chunksize=(512, 512, 2), chunktype=numpy.ndarray>
 Coordinates:
   * y        (y) int64 0 1 2 3 4 5 6 7 8 ... 503 504 505 506 507 508 509 510 511
   * x        (x) int64 0 1 2 3 4 5 6 7 8 ... 503 504 505 506 507 508 509 510 511
 Dimensions without coordinates: d0)

In [9]:
test_isotropic_ps_slope(other_dim_sizes=[10], chunks={'d0': 2}, dim_order=False)

/Users/aponte/.miniconda3/envs/equinox/lib/python3.7/site-packages/ipykernel_launcher.py:16: RuntimeWarning: divide by zero encountered in power
  app.launch_new_instance()


(<xarray.DataArray 'rechunk-merge-999eebe04b5f1b9861c484a59c791fc1' (d0: 10, freq_r: 127)>
 dask.array<getitem, shape=(10, 127), dtype=float64, chunksize=(2, 1), chunktype=numpy.ndarray>
 Coordinates:
   * freq_r          (freq_r) float64 0.002653 0.006041 0.009854 ... 0.4921 0.496
     freq_y_spacing  float64 0.001953
     freq_x_spacing  float64 0.001953
 Dimensions without coordinates: d0,
 <xarray.DataArray (d0: 10, y: 512, x: 512)>
 dask.array<xarray-<this-array>, shape=(10, 512, 512), dtype=float64, chunksize=(2, 512, 512), chunktype=numpy.ndarray>
 Coordinates:
   * y        (y) int64 0 1 2 3 4 5 6 7 8 ... 503 504 505 506 507 508 509 510 511
   * x        (x) int64 0 1 2 3 4 5 6 7 8 ... 503 504 505 506 507 508 509 510 511
 Dimensions without coordinates: d0)

In [10]:
test_isotropic_ps_slope(chunks={'y': None, 'x': None, 'd0': 2}, other_dim_sizes=[10], dim_order=True)

/Users/aponte/.miniconda3/envs/equinox/lib/python3.7/site-packages/ipykernel_launcher.py:16: RuntimeWarning: divide by zero encountered in power
  app.launch_new_instance()


(<xarray.DataArray 'rechunk-merge-7db83f14b7a8f8c86b349dab86f2137e' (d0: 10, freq_r: 127)>
 dask.array<getitem, shape=(10, 127), dtype=float64, chunksize=(2, 1), chunktype=numpy.ndarray>
 Coordinates:
   * freq_r          (freq_r) float64 0.002653 0.006041 0.009854 ... 0.4921 0.496
     freq_y_spacing  float64 0.001953
     freq_x_spacing  float64 0.001953
 Dimensions without coordinates: d0,
 <xarray.DataArray (y: 512, x: 512, d0: 10)>
 dask.array<xarray-<this-array>, shape=(512, 512, 10), dtype=float64, chunksize=(512, 512, 2), chunktype=numpy.ndarray>
 Coordinates:
   * y        (y) int64 0 1 2 3 4 5 6 7 8 ... 503 504 505 506 507 508 509 510 511
   * x        (x) int64 0 1 2 3 4 5 6 7 8 ... 503 504 505 506 507 508 509 510 511
 Dimensions without coordinates: d0)